In [1]:
import hail as hl
import pandas as pd
import numpy as np
import scipy.stats as stats
import os.path
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://10.151.24.211:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /Users/yanyul/Documents/repo/github/toy-example-of-hail-using-1kg/notebook/hail-20191212-1754-0.2.28-61941242c15d.log


In [2]:
variant = hl.read_table('output/variant_qc.ht')
mt = hl.read_matrix_table('data/1kg.mt')
print('Before filtering for variants:', mt.count())
mt = mt.filter_rows(hl.is_defined(variant[mt.locus, mt.alleles]))
print('After filtering for variants:', mt.count())

Before filtering for variants: (10961, 284)
After filtering for variants: (8777, 284)


In [3]:
type_dic = { f'trait_{i}' : hl.tfloat for i in range(20) }
pheno_table = hl.import_table('output/indiv_pheno.tsv', types = type_dic)

2019-12-12 17:54:19 Hail: INFO: Reading table with no type imputation
  Loading column 's' as type 'str' (type not specified)
  Loading column 'trait_0' as type 'float64' (user-specified)
  Loading column 'trait_1' as type 'float64' (user-specified)
  Loading column 'trait_2' as type 'float64' (user-specified)
  Loading column 'trait_3' as type 'float64' (user-specified)
  Loading column 'trait_4' as type 'float64' (user-specified)
  Loading column 'trait_5' as type 'float64' (user-specified)
  Loading column 'trait_6' as type 'float64' (user-specified)
  Loading column 'trait_7' as type 'float64' (user-specified)
  Loading column 'trait_8' as type 'float64' (user-specified)
  Loading column 'trait_9' as type 'float64' (user-specified)
  Loading column 'trait_10' as type 'float64' (user-specified)
  Loading column 'trait_11' as type 'float64' (user-specified)
  Loading column 'trait_12' as type 'float64' (user-specified)
  Loading column 'trait_13' as type 'float64' (user-specified)
  

### Comments go here
Here we take two random subsets of individuals and run two multi-trait GWASs.
The idea is to test the usage of `linear_regression_rows` in hail 0.2 in which `y` can be list of lists.

#### Before running the thing
We take a detour to create individual subsetting list. 

In [4]:
nsubset = 2
if not os.path.isfile('output/training_0.txt'): 
    def draw_subset(ntotal, ntarget, seed):
        if ntotal < ntarget:
            return None
        else:
            np.random.seed(seed)
            return np.random.choice(ntotal, ntarget, replace = False)
    def save_as_table(vec, f):
        o = open(f, 'w')
        for i in vec:
            o.write(i + '\n')
        o.close()
    indiv_list = np.array(mt.s.collect())
    subset_size = 200
    for i in range(nsubset):
        subset_idx = draw_subset(indiv_list.shape[0], subset_size, seed = i)
        training = indiv_list[subset_idx]
        save_as_table(training, f'output/training_{i}.txt')
        test = indiv_list[np.where(np.isin(np.array(range(indiv_list.shape[0])), subset_idx, invert = True))]
        save_as_table(test, f'output/test_{i}.txt')

#### OK, we can start to work on the GWAS pipeline

In [5]:
# loading phenotypes
pheno_full_table = pd.read_csv('output/indiv_pheno.tsv', sep = '\t')
target_trait_list = [ f'trait_{i}' for i in range(12) ]
my_pheno_list = []
for i in range(nsubset):
    pheno_set1 = pd.read_csv(f'output/training_{i}.txt', sep = '\t', header = None)
    pheno_subset = pheno_full_table.loc[pheno_full_table['s'].isin(pheno_set1[0].to_list())]
    pheno_subset = pheno_subset[['s'] + target_trait_list]
    pheno_subset_ht = hl.Table.from_pandas(pheno_subset, key = 's')
    my_pheno_list.append(pheno_subset_ht)

2019-12-12 17:54:21 Hail: INFO: Coerced sorted dataset
2019-12-12 17:54:22 Hail: INFO: Coerced sorted dataset


In [6]:
# loading covariates
type_dic = { f'covar_{i}': hl.tfloat for i in range(5) }
covar = hl.import_table('output/indiv_covar.tsv', types = type_dic, key = 's')

2019-12-12 17:54:22 Hail: INFO: Reading table with no type imputation
  Loading column 's' as type 'str' (type not specified)
  Loading column 'covar_0' as type 'float64' (user-specified)
  Loading column 'covar_1' as type 'float64' (user-specified)
  Loading column 'covar_2' as type 'float64' (user-specified)
  Loading column 'covar_3' as type 'float64' (user-specified)
  Loading column 'covar_4' as type 'float64' (user-specified)



In [7]:
# looks good, let's run GWAS
# annotate phenotypes
anno_pheno_expr = {
    f'subset_{i}': my_pheno_list[i][mt.s] for i in range(len(my_pheno_list))
}
mt = mt.annotate_cols(**anno_pheno_expr)
# annotate covariates
mt = mt.annotate_cols(covariate = covar[mt.s])

In [8]:
# and run linear_regression_rows
pheno_list = [ [ mt[f'subset_{i}'][j] for j in mt[f'subset_{i}'] ] for i in range(nsubset) ]
pheno_names = [ [ f'subset_{i}_x_{j}' for j in mt[f'subset_{i}'] ] for i in range(nsubset) ]
covar_list = [ mt.covariate[i] for i in list(mt.covariate.keys()) ]
gwas_out = hl.linear_regression_rows(
    y = pheno_list,
    x = mt.GT.n_alt_alleles(),
    covariates = covar_list + [1]
)

2019-12-12 17:54:23 Hail: WARN: 84 of 284 samples have a missing phenotype or covariate.
2019-12-12 17:54:23 Hail: WARN: 84 of 284 samples have a missing phenotype or covariate.
2019-12-12 17:54:23 Hail: INFO: linear_regression_rows[0]: running on 200 samples for 12 response variables y,
    with input variable x, and 6 additional covariates...
2019-12-12 17:54:24 Hail: INFO: linear_regression_rows[1]: running on 200 samples for 12 response variables y,
    with input variable x, and 6 additional covariates...


In [ ]:
gwas_out = gwas_out.annotate_globals(phenotypes = pheno_names)
gwas_out.describe()

----------------------------------------
Global fields:
    'phenotypes': array<array<str>> 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'n': array<int32> 
    'sum_x': array<float64> 
    'y_transpose_x': array<array<float64>> 
    'beta': array<array<float64>> 
    'standard_error': array<array<float64>> 
    't_stat': array<array<float64>> 
    'p_value': array<array<float64>> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


#### OK, the last step is to format results into tables by trait
Here I mostly follow routine in [here](https://github.com/Nealelab/UK_Biobank_GWAS/blob/95ac260a5d4cf9c40effff13fa33fb95ed825e2a/0.2/run_regressions.biomarkers.py#L54) and [here](https://github.com/Nealelab/UK_Biobank_GWAS/blob/95ac260a5d4cf9c40effff13fa33fb95ed825e2a/0.2/export_results.biomarkers.py)

In [ ]:
gwas_out = gwas_out.annotate( 
    variant = hl.delimit(
        hl.array([
            gwas_out['locus'].contig,
            hl.str(gwas_out['locus'].position),
            gwas_out['alleles'][0],
            gwas_out['alleles'][1]
        ]), 
    delimiter = ':')
)
gwas_out = gwas_out.key_by('variant')
# gwas_out = gwas_out.repartition(116)
# gwas_out = gwas_out.cache()

phenotypes = gwas_out['phenotypes'].collect()[0]
for i, subset in enumerate(phenotypes):
    for j, trait in enumerate(subset):
#         trait_name = trait.split('_x_')[-1]
        ht_export = gwas_out.annotate(
            n_complete_samples = gwas_out['n'][i],
            AC = gwas_out['sum_x'][i],
            ytx = gwas_out['y_transpose_x'][i][j],
            beta = gwas_out['beta'][i][j],
            se = gwas_out['standard_error'][i][j],
            tstat = gwas_out['t_stat'][i][j],
            pval = gwas_out['p_value'][i][j])
        ht_export = ht_export.annotate(
            AF = ht_export['AC'] / (2 * ht_export['n_complete_samples']))
        ht_export = ht_export.annotate(
            minor_AF = hl.cond(ht_export['AF'] <= 0.5, ht_export['AF'], 1.0 - ht_export['AF']),
            minor_allele = hl.cond(ht_export['AF'] <= 0.5, ht_export['alleles'][1], ht_export['alleles'][0]))
        ht_export = ht_export.annotate(
            low_confidence_variant=ht_export['minor_AF'] < 0.001)
        ht_export = ht_export.select(
            'minor_allele',
            'minor_AF',
            'low_confidence_variant',
            'n_complete_samples',
            'AC',
            'ytx',
            'beta',
            'se',
            'tstat',
            'pval')
        ht_export.export(f'output/gwas_{trait}.tsv')

2019-12-12 17:54:25 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-12 17:54:27 Hail: INFO: merging 2 files totalling 914.4K...
2019-12-12 17:54:27 Hail: INFO: while writing:
    output/gwas_subset_0_x_trait_0.tsv
  merge time: 21.489ms
2019-12-12 17:54:27 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-12 17:54:28 Hail: INFO: merging 2 files totalling 905.8K...
2019-12-12 17:54:28 Hail: INFO: while writing:
    output/gwas_subset_0_x_trait_1.tsv
  merge time: 21.347ms
2019-12-12 17:54:29 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-12 17:54:30 Hail: INFO: merging 2 files totalling 905.8K...
2019-12-12 17:54:30 Hail: INFO: while writing:
    output/gwas_subset_0_x_trait_2.tsv
  merge time: 15.068ms
2019-12-12 17:54:30 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-12 17:54:31 Hail: INFO: merging 2 files totalling 914.5K...
2019-12-12 17:54:31 Hail: INFO: while writing:
    output/gwas_subset_0_x_trait_3.tsv

In [ ]:
# # gwas_out.annotate_globals()
# phenotypes = gwas_out['phenotypes'].collect()[0]
# phenotypes
# gwas_out['n'][i].show()
# gwas_out['y_transpose_x'][i][j].show()